<a href="https://colab.research.google.com/github/suann124/AIinTeaching_Archive/blob/main/lectures/Linear_Systems/matlab/Linear_Systems_MATLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear System MATLAB script

This page allows you to download the MATLAB scripts for simulations in First-order Systems Lecture.

Just run the cell of the script you want or "Run all" to download everything and they will be saved under "Files".


In [1]:
# @title 1. Example: the relation between the state evolution and matrix exponential.
%%writefile matrix_exponential_phase_plane_example.m

clear; close all; clc;

% ==============================
% User inputs
% ==============================
A = [-0.1, -1.0;
      1.0, -0.1];

x0 = [0.5; 0.75];

t0 = 0.0;
tf = 10.0;
num_points = 600;

% ==============================
% ODE solution
% ==============================
t_eval = linspace(t0, tf, num_points);
sol = ode45(@(t,x) A*x, [t0 tf], x0);
x = deval(sol, t_eval);

x1 = x(1,:);
x2 = x(2,:);

t_mark = 1.0;
x_t1 = deval(sol, t_mark);

% ==============================
% Plot
% ==============================
figure('Position',[100 100 700 700],'Color','w');
hold on;

% trajectory
plot(x1, x2, 'LineWidth', 2, 'Color', [0.2 0.5 0.9]);

% blue triangle arrows along trajectory using filled patches
arrow_indices = round(linspace(40, num_points-40, 12));
arrow_size = 0.04;
for k = arrow_indices
    dx = x1(k+3)-x1(k-3);
    dy = x2(k+3)-x2(k-3);
    angle = atan2(dy, dx);

    tri_x = x1(k) + arrow_size * [cos(angle), cos(angle+2.5), cos(angle-2.5)];
    tri_y = x2(k) + arrow_size * [sin(angle), sin(angle+2.5), sin(angle-2.5)];

    fill(tri_x, tri_y, [0.2 0.5 0.9], 'EdgeColor', [0.2 0.5 0.9]);
end

% initial point x0 - light blue filled circle
plot(x0(1), x0(2), 'o', 'MarkerSize', 10, 'MarkerFaceColor', [0.6 0.8 1.0], ...
    'MarkerEdgeColor', [0.4 0.6 0.8], 'LineWidth', 1);
text(x0(1)+0.06, x0(2)+0.0, '$x_0$', 'Interpreter', 'latex', 'FontSize', 14);

% A x0 - red arrow
Ax0 = A*x0;
quiver(x0(1), x0(2), Ax0(1), Ax0(2), 0, ...
    'Color', 'r', 'LineWidth', 2, 'MaxHeadSize', 0.5);
text(x0(1)+Ax0(1)-0.08, x0(2)+Ax0(2)+0.1, '$Ax_0$', ...
    'Interpreter', 'latex', 'FontSize', 14, 'Color', 'r');

% x(1) - orange filled circle
plot(x_t1(1), x_t1(2), 'o', 'MarkerSize', 10, 'MarkerFaceColor', [1.0 0.6 0.2], ...
    'MarkerEdgeColor', [0.9 0.5 0.1], 'LineWidth', 1);
text(x_t1(1)-0.08, x_t1(2)-0.12, '$x(1)=e^{A\cdot1}x_0$', ...
    'Interpreter', 'latex', 'FontSize', 14);

% A x(1) - red arrow
Ax1 = A*x_t1;
quiver(x_t1(1), x_t1(2), Ax1(1), Ax1(2), 0, ...
    'Color', 'r', 'LineWidth', 2, 'MaxHeadSize', 0.5);
text(x_t1(1)+Ax1(1)-0.12, x_t1(2)+Ax1(2)+0.15, '$Ax(1)$', ...
    'Interpreter', 'latex', 'FontSize', 14, 'Color', 'r');

% labels & title
xlabel('$x_1$', 'Interpreter', 'latex', 'FontSize', 14);
ylabel('$x_2$', 'Interpreter', 'latex', 'FontSize', 14);
title('Solution of $\dot{x}=Ax$ in the phase plane', 'Interpreter', 'latex', 'FontSize', 16);

% axis settings with padding
axis equal;
xlim([-1.15, 0.75]);
ylim([-0.75, 1.4]);
grid on;
box on;
set(gca, 'FontSize', 12);

hold off;


Writing matrix_exponential_phase_plane_example.m


In [ ]:
# @title 2. Simulation of the free response

%%writefile free_response_simulation.m
%
% This script reproduces the Python simulation of free responses
% of 2D linear systems, including phase portraits and time responses.

clear; close all; clc;

% ==============================
% Helper: simulate xdot = A x
% ==============================
function [t_eval, traj] = simulate_linear_system(A, x0, t_span, num_points)
    if nargin < 4
        num_points = 800;
    end
    t_eval = linspace(t_span(1), t_span(2), num_points);
    sol = ode45(@(t,x) A*x, t_span, x0);
    traj = deval(sol, t_eval);  % size 2 x N
end

% ==============================
% System matrices
% ==============================

% Case 1: diagonal stable node
A1 = [-1.0,  0.0;
       0.0, -0.4];

% Case 2: skew-symmetric center
beta = 2.0;
A2 = [ 0.0,  beta;
      -beta, 0.0];

% Case 3: stable spiral
alpha = -0.3;
beta  =  2.0;
A3 = [ alpha,  beta;
      -beta, alpha];

% ==============================
% Initial conditions
% ==============================

% Case 1 & 3: quadrants
inits_quadrants = {
    [ 1.0;  1.0]
    [-1.0;  1.0]
    [-1.0; -1.0]
    [ 1.0; -1.0]
};

% Case 2: different radii
inits_radii = {
    [0.5; 0.0]
    [1.0; 0.0]
    [1.5; 0.0]
    [2.0; 0.0]
};

cases = {
    'Case 1', A1, inits_quadrants, [0.0 10.0];
    'Case 2', A2, inits_radii,     [0.0 10.0];
    'Case 3', A3, inits_quadrants, [0.0 12.0];
};

% ==============================
% Plot: 2 rows x 3 columns
% ==============================
figure('Position',[100 100 1700 900],'Color','w');

for col = 1:3
    title_str = cases{col,1};
    A         = cases{col,2};
    inits     = cases{col,3};
    t_span   = cases{col,4};

    % ---------- Row 1: Phase portrait ----------
    ax_phase = subplot(2,3,col);
    hold on;

    for k = 1:length(inits)
        x0 = inits{k};
        [t, traj] = simulate_linear_system(A, x0, t_span);
        x1 = traj(1,:);
        x2 = traj(2,:);

        plot(x1, x2, 'LineWidth', 2);
        scatter(x0(1), x0(2), 40, 'filled');

        % Direction arrows
        arrow_indices = round(linspace(100, length(x1)-100, 5));
        for idx = arrow_indices
            dx = x1(idx+1) - x1(idx);
            dy = x2(idx+1) - x2(idx);
            quiver(x1(idx), x2(idx), dx, dy, 0, ...
                'Color','k','LineWidth',1.2,'MaxHeadSize',0.6);
        end
    end

    title(title_str);
    xlabel('$x_1$','Interpreter','latex');
    ylabel('$x_2$','Interpreter','latex');
    grid on;
    axis equal;
    box on;

    xlim_current = xlim;
    ylim_current = ylim;

    x_pad = 0.08 * (xlim_current(2) - xlim_current(1));
    y_pad = 0.08 * (ylim_current(2) - ylim_current(1));

    xlim([xlim_current(1) - x_pad, xlim_current(2) + x_pad]);
    ylim([ylim_current(1) - y_pad, ylim_current(2) + y_pad]);

    hold off;

    % ---------- Row 2: Time responses ----------
    ax_time = subplot(2,3,col+3);
    hold on;

    x0 = inits{1};
    [t, traj] = simulate_linear_system(A, x0, t_span);
    x1 = traj(1,:);
    x2 = traj(2,:);

    plot(t, x1, 'LineWidth', 2, 'DisplayName','$x_1(t)$');
    plot(t, x2, 'LineWidth', 2, 'DisplayName','$x_2(t)$');

    xlabel('$t$','Interpreter','latex');
    ylabel('$x_i(t)$','Interpreter','latex');
    grid on;
    legend('Interpreter','latex','Location','best');
    box on;

    xlim_current = xlim;
    ylim_current = ylim;

    x_pad = 0.08 * (xlim_current(2) - xlim_current(1));
    y_pad = 0.08 * (ylim_current(2) - ylim_current(1));

    xlim([xlim_current(1), xlim_current(2) + x_pad]);
    ylim([ylim_current(1) - y_pad, ylim_current(2) + y_pad]);

    hold off;
end



In [ ]:
# @title 3. Example: effect of diagonalization

%%writefile diagonalization_effect.m
%
% MATLAB version of the Python example:
% * Two side-by-side phase planes
% * Normalized vector field (unit direction arrows)
% * 12 trajectories from points on a circle
% * Integrate forward + backward in time and concatenate

clear; close all; clc;

% ----------------------------
% Matrices
% ----------------------------
A = [ 0.0,  1.0;
     -2.0, -3.0];

Atilde = [-2.0,  0.0;
           0.0, -1.0];

% ----------------------------
% Initial conditions: 12 points on a circle
% ----------------------------
m = 12;
r = 3.0;
angles = linspace(0, 2*pi, m+1);
angles(end) = [];
x0s = [r*cos(angles(:)), r*sin(angles(:))];  % m-by-2

% ----------------------------
% Plot
% ----------------------------
figure('Position',[100 100 1200 500],'Color','w');

% Left subplot
ax1 = subplot(1,2,1);
title('System 1:  xdot = A x,   A = [[0, 1], [-2, -3]]','Interpreter','none');
xlim([-4 4]); ylim([-4 4]);
axis equal;
grid on; ax1.GridAlpha = 0.25; box on;
hold on;
plot_vector_field(ax1, A, [-4 4], [-4 4], 25, 6, 0.006);
plot_trajectories(ax1, A, x0s, 6.0, 700);

% Matplotlib-style padding (after axis equal)
drawnow;
pad_axes(ax1, 0.08);

hold off;

% Right subplot
ax2 = subplot(1,2,2);
title('System 2:  xtildedot = Atilde*xtilde,   Atilde = [[-2, 0], [0, -1]]','Interpreter','none');
xlim([-4 4]); ylim([-4 4]);
axis equal;
grid on; ax2.GridAlpha = 0.25; box on;
hold on;
plot_vector_field(ax2, Atilde, [-4 4], [-4 4], 25, 6, 0.006);
plot_trajectories(ax2, Atilde, x0s, 6.0, 700);

% Matplotlib-style padding (after axis equal)
drawnow;
pad_axes(ax2, 0.08);

hold off;

% ============================
% Local functions
% ============================

function plot_vector_field(ax, A, xlim_vec, ylim_vec, ngrid, scale, width)
% Normalized vector field arrows (direction only), similar to Python:
% quiver(X,Y,U/mag,W/mag, scale=scale, width=width)

    xs = linspace(xlim_vec(1), xlim_vec(2), ngrid);
    ys = linspace(ylim_vec(1), ylim_vec(2), ngrid);
    [X, Y] = meshgrid(xs, ys);

    V = A * [X(:)'; Y(:)'];
    U = reshape(V(1,:), size(X));
    W = reshape(V(2,:), size(Y));

    mag = sqrt(U.^2 + W.^2);
    mag = max(mag, 1e-8);

    Uu = U ./ mag;
    Wu = W ./ mag;

    % MATLAB quiver scaling is different from Matplotlib.
    % Use AutoScale ON and tune AutoScaleFactor to emulate "scale=6".
    hq = quiver(ax, X, Y, Uu, Wu, 'k', 'LineWidth', 0.5);
    hq.AutoScale = 'on';
    hq.AutoScaleFactor = 1/scale;   % heuristic match to Matplotlib's scale
end

function plot_trajectories(ax, A, x0s, T, n_eval)
% Integrate forward and backward and concatenate, like Python code.

    t_eval_f = linspace(0.0,  T, n_eval);
    t_eval_b = linspace(0.0, -T, n_eval);

    for i = 1:size(x0s,1)
        x0 = x0s(i,:)';

        sol_f = ode45(@(t,x) A*x, [0.0  T], x0);
        sol_b = ode45(@(t,x) A*x, [0.0 -T], x0);

        xf = deval(sol_f, t_eval_f);  % 2-by-n_eval
        xb = deval(sol_b, t_eval_b);  % 2-by-n_eval

        % reverse backward so it's past -> x0
        xb = xb(:, end:-1:1);

        % skip duplicate at t=0 from forward part
        traj = [xb, xf(:, 2:end)];

        plot(ax, traj(1,:), traj(2,:), 'LineWidth', 2);
        plot(ax, x0(1), x0(2), 'o', 'MarkerSize', 4, 'MarkerFaceColor', 'k', 'MarkerEdgeColor', 'k');
    end
end

function pad_axes(ax, frac)
% Add Matplotlib-style padding to current axis limits.
% Call after plotting (and after axis equal if used).

    axes(ax); %#ok<LAXES>
    xl = xlim;
    yl = ylim;

    dx = xl(2) - xl(1);
    dy = yl(2) - yl(1);

    if dx == 0, dx = 1; end
    if dy == 0, dy = 1; end

    x_pad = frac * dx;
    y_pad = frac * dy;

    xlim([xl(1) - x_pad, xl(2) + x_pad]);
    ylim([yl(1) - y_pad, yl(2) + y_pad]);
end


In [ ]:
# @title 4. The long-term behavior of $t^k e^{\lambda_i t}$

%%writefile long_term_behavior_tk_exp.m

% MATLAB version of the Python example illustrating
% the decay of t^k e^{lambda t} for lambda < 0.

clear; close all; clc;

% ----------------------------
% Parameter
% ----------------------------
lam = -1.0;

t = linspace(0, 20, 2000);

y0 = exp(lam * t);
y1 = t .* y0;
y2 = t.^2 .* y0;
y3 = t.^3 .* y0;
y4 = t.^4 .* y0;

% ----------------------------
% Plot
% ----------------------------
figure('Position',[100 100 1000 600],'Color','w');
hold on;

plot(t, y0, 'LineWidth', 2, 'DisplayName', '$e^{\lambda_i t}$');
plot(t, y1, 'LineWidth', 2, 'DisplayName', '$t e^{\lambda_i t}$');
plot(t, y2, 'LineWidth', 2, 'DisplayName', '$t^2 e^{\lambda_i t}$');
plot(t, y3, 'LineWidth', 2, 'DisplayName', '$t^3 e^{\lambda_i t}$');
plot(t, y4, 'LineWidth', 2, 'DisplayName', '$t^4 e^{\lambda_i t}$');

title(sprintf('Terms $t^k e^{\\lambda_i t}$ for $\\lambda_i = %.1f$', lam), ...
    'Interpreter','latex');

xlabel('$t$','Interpreter','latex');
ylabel('magnitude','Interpreter','latex');

grid on;
legend('Interpreter','latex','Location','best');
box on;

ylim([0, max(ylim)]);   % match Python: bottom = 0

% ----------------------------
% Matplotlib-style padding
% ----------------------------
drawnow;

xl = xlim;
yl = ylim;

x_pad = 0.08 * (xl(2) - xl(1));
y_pad = 0.08 * (yl(2) - yl(1));

xlim([xl(1), xl(2) + x_pad]);
ylim([yl(1), yl(2) + y_pad]);   % only pad upward (same intent as Python)

hold off;



In [ ]:
# @title 5. The modes of a linear system

%%writefile modes_of_linear_system.m
%
% MATLAB version of the Python example:
% * Compute eigenvalues/eigenvectors
% * Build initial conditions using eigenvectors (incl. complex pair)
% * Evolve x(t) via diagonalization: x(t) = P * exp(Lambda t) * P^{-1} * x0
% * Plot x_i(t) for 3 different initial conditions (1x3 subplots)
%
% Note: Results depend on eigenvector ordering (same caveat as NumPy).

clear; close all; clc;

% ------------------------------------------------------------
% Matrix A (eigenvalues: -1, -4, -0.2 ± 3i)
% ------------------------------------------------------------
A = [
    -1.2,  0.8, -2.0, -1.0;
    -1.0, -2.0, -1.0, -3.0;
     0.8,  1.8, -1.2, -2.0;
    -1.0, -1.0,  3.0, -2.0
];

% ------------------------------------------------------------
% Eigenvalues, eigenvectors, diagonalization
% ------------------------------------------------------------
[P, D] = eig(A);                 % A*P = P*D
eigvals = diag(D);               % 4x1 (possibly complex)
Pinv = inv(P);

disp('Eigenvalues:');
disp(eigvals);

% Separate real and complex eigenvectors (by eigenvalue imag part)
tol = 1e-8;
real_inds = find(abs(imag(eigvals)) < tol);
cplx_inds = find(abs(imag(eigvals)) >= tol);

% Grab vectors (same logic as Python)
v1 = P(:, real_inds(1));
v2 = P(:, real_inds(2));
v3 = P(:, cplx_inds(1));
v4 = P(:, cplx_inds(2));

% ------------------------------------------------------------
% Initial conditions (first three only)
% ------------------------------------------------------------
x0_list = {
    v1 + 0.1*v2 + 0.1*v3 + 0.1*v4
    0.1*v1 + v2 + 0.1*v3 + 0.1*v4
    0.1*v1 + 0.1*v2 + v3 + v4
};

labels = {
    '$x_0 = v_1 + 0.1v_2 + 0.1v_3 + 0.1v_4$'
    '$x_0 = 0.1v_1 + v_2 + 0.1v_3 + 0.1v_4$'
    '$x_0 = 0.1v_1 + 0.1v_2 + v_3 + v_4$'
};

% ------------------------------------------------------------
% State evolution via diagonalization
% x(t) = P exp(diag(eigvals) t) P^{-1} x0
% ------------------------------------------------------------
lam = eigvals;   % 4x1 complex

% Time grid
t = linspace(0, 15, 1200);  % 1xN

% ------------------------------------------------------------
% Plot x_i(t) for the three initial conditions (side by side)
% ------------------------------------------------------------
figure('Position',[100 100 1600 450],'Color','w');

for k = 1:3
    ax = subplot(1,3,k);
    hold on;

    x0 = x0_list{k};

    % z0 = P^{-1} x0
    z0 = Pinv * x0;                 % 4x1 complex
    % z(t) = exp(lam t) .* z0  (elementwise per mode)
    expLt = exp(lam * t);           % (4x1)*(1xN) -> 4xN
    zt = expLt .* (z0 * ones(1, numel(t)));  % 4xN
    Xt = P * zt;                    % 4xN complex
    X = real(Xt);                   % mimic np.real_if_close(...).real

    plot(t, X(1,:), 'LineWidth', 1.5, 'DisplayName', '$x_1(t)$');
    plot(t, X(2,:), 'LineWidth', 1.5, 'DisplayName', '$x_2(t)$');
    plot(t, X(3,:), 'LineWidth', 1.5, 'DisplayName', '$x_3(t)$');
    plot(t, X(4,:), 'LineWidth', 1.5, 'DisplayName', '$x_4(t)$');

    title(labels{k}, 'Interpreter','latex');
    xlabel('t');
    ylabel('state value');
    grid on;
    ax.GridAlpha = 0.3;
    legend('Interpreter','latex','FontSize',9,'Location','best');
    box on;

    % Matplotlib-style padding (per your rule)
    drawnow;
    xl = xlim; yl = ylim;
    x_pad = 0.08 * (xl(2) - xl(1));
    y_pad = 0.08 * (yl(2) - yl(1));
    xlim([xl(1), xl(2) + x_pad]);
    ylim([yl(1) - y_pad, yl(2) + y_pad]);

    hold off;
end


In [ ]:
# @title 6. Example: Second-order system with real, distinct eigenvalues

%%writefile second_order_real_distinct_eigs.m
%
% MATLAB version of the Python example:
% * Construct A = P diag([lam_slow, lam_fast]) P^{-1} with eigenvectors ~80° apart
% * Compute eig(A), sort so lambda_1 is least-negative (largest real part)
% * Plot eigenvector direction lines and label eigenvalues
% * Simulate trajectories using expm(A*t) from 12 initial points on a circle
% * Apply Matplotlib-style axis padding (per your rule)

clear; close all; clc;

% ------------------------------------------------------------
% Build a 2x2 system with well-separated eigenvectors
% ------------------------------------------------------------
lam_slow = -0.3;
lam_fast = -2.0;

theta = deg2rad(80.0);
v1_des = [1.0; 0.0];
v2_des = [cos(theta); sin(theta)];

P_des = [v1_des, v2_des];
A = P_des * diag([lam_slow, lam_fast]) / P_des;   % same as P*D*inv(P)

% Eigen-decomposition (for labeling and plotting the true eigenvectors)
[V, D] = eig(A);
lam = diag(D);

% Sort so that lambda_1 is the "less negative" (largest real part)
[~, order] = sort(real(lam), 'descend');
lam = lam(order);
V = V(:, order);

% Normalize eigenvectors for plotting
v1 = real(V(:,1));
v1 = v1 / (norm(v1) + 1e-12);

v2 = real(V(:,2));
v2 = v2 / (norm(v2) + 1e-12);

% ------------------------------------------------------------
% Initial conditions: points on a circle
% ------------------------------------------------------------
m = 12;
r = 3.0;
angles = linspace(0, 2*pi, m+1);
angles(end) = [];
x0s = [r*cos(angles(:)), r*sin(angles(:))];   % m-by-2

% Time grid
t = linspace(0, 15, 700);

% ------------------------------------------------------------
% Plot: eigenvector directions + many trajectories
% ------------------------------------------------------------
figure('Position',[100 100 700 700],'Color','w');
ax = gca;
hold on;
axis equal;
grid on;
ax.GridAlpha = 0.3;

% Draw eigenvector direction lines through origin
s = 4.2;
plot([-s*v1(1), s*v1(1)], [-s*v1(2), s*v1(2)], '--', 'LineWidth', 3);
plot([-s*v2(1), s*v2(1)], [-s*v2(2), s*v2(2)], '--', 'LineWidth', 2);

% Label eigenvectors with eigenvalues
text(1.05*s*v1(1), 1.05*s*v1(2), sprintf('$\\lambda_1 = %.2f$', real(lam(1))), ...
    'Interpreter','latex','FontSize',12);
text(1.05*s*v2(1), 1.05*s*v2(2), sprintf('$\\lambda_2 = %.2f$', real(lam(2))), ...
    'Interpreter','latex','FontSize',12);

% Plot trajectories from multiple initial conditions using expm(A*t)
for i = 1:size(x0s,1)
    x0 = x0s(i,:)';

    X = zeros(2, numel(t));
    for k = 1:numel(t)
        X(:,k) = expm(A * t(k)) * x0;
    end

    plot(X(1,:), X(2,:), 'LineWidth', 2);
    plot(x0(1), x0(2), 'o', 'MarkerSize', 4, 'MarkerFaceColor', 'k', 'MarkerEdgeColor', 'k');
end

% Origin
plot(0, 0, 'k+', 'MarkerSize', 10, 'LineWidth', 1.5);

xlim([-4.5 4.5]);
ylim([-4.5 4.5]);

xlabel('x1');
ylabel('x2');
title('Trajectories align with the eigenvector of the least-negative eigenvalue');

box on;

% Matplotlib-style padding (after axis equal / limits)
drawnow;
xl = xlim; yl = ylim;
x_pad = 0.08 * (xl(2) - xl(1));
y_pad = 0.08 * (yl(2) - yl(1));
xlim([xl(1) - x_pad, xl(2) + x_pad]);
ylim([yl(1) - y_pad, yl(2) + y_pad]);

hold off;


In [ ]:
# @title 7. Example: Second-order system with complex eigenvalues

%%writefile second_order_complex_eigs.m
%
% MATLAB version of the Python example:
% * A = [[sigma, -omega],[omega, sigma]]
% * Closed-form solution: x(t) = e^{sigma t} R(omega t) x0
% * Plot one component with envelope ±||x0|| e^{sigma t}
% * 3x2 subplot grid (sigmas x omegas), shared x-axis style
% * Matplotlib-style padding on every subplot (per your rule)

clear; close all; clc;

% ------------------------------------------------------------
% Parameters
% ------------------------------------------------------------
sigmas = [-0.1, -0.5, 0.5];   % real parts
omegas = [1.0, 5.0];          % imaginary parts (angular frequencies)

T = 20.0;
N = 2000;
t = linspace(0, T, N);

x0 = [1.0; 0.0];              % initial condition
component_to_plot = 1;        % 1 -> x1(t), 2 -> x2(t)   (MATLAB is 1-indexed)

% ------------------------------------------------------------
% Figure + subplots
% ------------------------------------------------------------
figure('Position',[100 100 1200 700],'Color','w');

nR = numel(sigmas);
nC = numel(omegas);

axes_handles = gobjects(nR, nC);

for c = 1:nC
    omega = omegas(c);

    for r = 1:nR
        sigma = sigmas(r);

        % A = [sigma, -omega; omega, sigma];
        % Closed-form solution
        ct = cos(omega * t);
        st = sin(omega * t);
        exp_env = exp(sigma * t);

        x1 = exp_env .* ( ct * x0(1) - st * x0(2) );
        x2 = exp_env .* ( st * x0(1) + ct * x0(2) );

        if component_to_plot == 1
            x_comp = x1;
        else
            x_comp = x2;
        end

        % Envelope ±||x0|| e^{sigma t}
        env = norm(x0) * exp_env;

        % subplot index (row-major like Python)
        ax = subplot(nR, nC, (r-1)*nC + c);
        axes_handles(r,c) = ax;
        hold on;

        plot(t, x_comp, 'LineWidth', 2);
        plot(t,  env, 'r--', 'LineWidth', 1.5);
        plot(t, -env, 'r--', 'LineWidth', 1.5);

        grid on;
        ax.GridAlpha = 0.25;
        box on;

        % Titles / labels
        if r == 1
            title(sprintf('$\\omega = %.1f$', omega), 'Interpreter','latex', 'FontSize', 13);
        end
        if c == 1
            ylabel(sprintf('$x_{%d}(t)$', component_to_plot), 'Interpreter','latex');
        end
        if r == nR
            xlabel('t');
        end

        % Annotations at axes-relative coordinates (like transform=ax.transAxes)
        text(0.62, 0.75, sprintf('$\\sigma = %g$', sigma), ...
            'Units','normalized', 'Interpreter','latex', 'FontSize', 12);

        % Matplotlib-style padding per subplot
        drawnow;
        xl = xlim; yl = ylim;
        x_pad = 0.08 * (xl(2) - xl(1));
        y_pad = 0.08 * (yl(2) - yl(1));
        xlim([xl(1), xl(2) + x_pad]);
        ylim([yl(1) - y_pad, yl(2) + y_pad]);

        hold off;
    end
end

% Overall title (like fig.suptitle)
sgtitle('Effect of $\sigma$ (growth/decay) and $\omega$ (oscillation frequency) for $x'' = Ax$', ...
    'Interpreter','latex', 'FontSize', 14);


In [ ]:
# @title 8. Example: Matching the eigenvalue locations to free response for second-order systems

%%writefile match_eigs_to_free_response.m
%
% MATLAB version of the Python example:
% * Define 9 eigenvalue locations (Re, Im)
% * Build A matrices via similarity transform: A = P*A0*P^{-1}
%   - complex lambda -> A0 = [a, -b; b, a]
%   - real lambda    -> A0 = diag([lambda, -10])
% * Random permutation (seed=7) to place systems in a 3x3 grid
% * Plot eigenvalue locations (top row)
% * Plot 3x3 time responses from x0 = (1,1) using expm(A*t)
% * Matplotlib-style axis padding added everywhere (per your rule)

clear; close all; clc;

% ------------------------------------------------------------
% Updated eigenvalue locations (real, imaginary)
% ------------------------------------------------------------
eigs_tbl = [ ...
    1, -1.5,  5.0;
    2,  0.0,  2.0;
    3,  0.0,  5.0;
    4,  1.5,  5.0;
    5, -10.0, 0.0;
    6, -1.0,  0.0;
    7,  0.0,  0.0;
    8,  1.0,  0.0;
    9,  3.0,  0.0];

labels = eigs_tbl(:,1);
re_list = eigs_tbl(:,2);
im_list = eigs_tbl(:,3);

% ------------------------------------------------------------
% Build A matrices:
% - complex lambda -> conjugate pair
% - real lambda -> second eigenvalue at -10
% ------------------------------------------------------------
P = [1.0, 1.0;
     0.4, 1.2];
Pinv = inv(P);

A_list = cell(9,1);
lam_list = complex(zeros(9,1));

for k = 1:9
    lam = complex(re_list(k), im_list(k));
    lam_list(k) = lam;

    if abs(imag(lam)) > 1e-12
        a = real(lam);
        b = imag(lam);
        A0 = [ a, -b;
               b,  a ];
    else
        A0 = diag([real(lam), -10.0]);
    end

    A_list{k} = P * A0 * Pinv;
end

% ------------------------------------------------------------
% Random placement in 3x3 grid (unlabeled), seed=7
% Python used rng = default_rng(7); perm = rng.permutation(9)
% Here we use MATLAB rng(7) and randperm(9) for a deterministic shuffle.
% ------------------------------------------------------------
rng(7);
perm = randperm(9);   % 1..9 indices

% ------------------------------------------------------------
% Simulate xdot = A x from x0 = (1,1) using expm
% ------------------------------------------------------------
t = linspace(0, 5, 1000);
x0 = [1.0; 1.0];

% ------------------------------------------------------------
% Figure layout: top eigenvalue plot + 3x3 time responses
% ------------------------------------------------------------
fig = figure('Position',[100 100 1400 1200],'Color','w');

% Use tiledlayout to mimic gridspec (4 rows x 3 cols)
% Row 1 spans all 3 columns, rows 2-4 are 3x3
tl = tiledlayout(4, 3, 'TileSpacing','compact', 'Padding','compact');

% -------------------------
% Top: eigenvalue locations
% -------------------------
ax0 = nexttile(tl, [1 3]);
hold on;

% Axes lines
plot([xlim(ax0)], [0 0]); %#ok<NBRAK> (placeholder; will reset below)

% Draw axes lines explicitly
plot([-12 4], [0 0], 'b', 'LineWidth', 2);
plot([0 0], [-6 6], 'b', 'LineWidth', 2);

for k = 1:9
    re = re_list(k);
    im = im_list(k);
    lab = labels(k);

    plot(re, im, 'rx', 'MarkerSize', 10, 'LineWidth', 2);
    text(re + 0.12, im + 0.12, sprintf('%d', lab), ...
        'Color','r', 'FontSize', 12);
end

xlabel('real', 'FontSize', 12);
ylabel('imaginary', 'FontSize', 12);

set(ax0, 'XTick', [], 'YTick', []);
xlim([-14 6]);
ylim([-8 8]);
grid on;
ax0.GridAlpha = 0.3;
box on;

% Matplotlib-style padding (after axis equal / limits)
drawnow;
pad_axes(ax0, 0.08);

hold off;

% -------------------------
% Bottom: 3x3 time responses
% -------------------------
for j = 1:9
    ax = nexttile(tl);
    hold on;

    idx = perm(j);        % 1..9
    A = A_list{idx};

    X = simulate_expm(A, t, x0);   % 2xN

    plot(t, X(1,:), 'LineWidth', 2);
    plot(t, X(2,:), 'LineWidth', 2);

    grid on;
    ax.GridAlpha = 0.3;
    box on;

    xlim([0 5]);
    xlabel('t');
    ylabel('state');

    % Matplotlib-style padding (per subplot)
    drawnow;
    pad_axes(ax, 0.08);

    hold off;
end

% ============================
% Local functions
% ============================

function X = simulate_expm(A, t, x0)
% X(:,i) = expm(A*t(i)) * x0
    N = numel(t);
    X = zeros(2, N);
    for i = 1:N
        X(:, i) = expm(A * t(i)) * x0;
    end
end

function pad_axes(ax, frac)
% Add Matplotlib-style padding to current axis limits.
% Call after plotting (and after axis equal if used).

    axes(ax); %#ok<LAXES>
    xl = xlim;
    yl = ylim;

    dx = xl(2) - xl(1);
    dy = yl(2) - yl(1);

    if dx == 0, dx = 1; end
    if dy == 0, dy = 1; end

    x_pad = frac * dx;
    y_pad = frac * dy;

    xlim([xl(1), xl(2) + x_pad]);
    ylim([yl(1) - y_pad, yl(2) + y_pad]);
end



In [ ]:
# @title 9. Example: Effect of damping ratio and natural frequency

%%writefile damping_ratio_natural_frequency.m
%
% MATLAB version of the Python example:
% Second-order homogeneous system:
%   y¨ + 2 ζ ω_n y˙ + ω_n^2 y = 0
% IC: y(0)=1, y˙(0)=0
%
% Two panels:
% * Left: vary damping ratio ζ (with a highlighted ζ=1 curve)
% * Right: vary natural frequency ω_n (with fixed ζ)
%
% Matplotlib-style axis padding applied (per your rule).

clear; close all; clc;

% ------------------------------------------------------------
% Time grid
% ------------------------------------------------------------
T = 20.0;
t = linspace(0, T, 2000);

% ------------------------------------------------------------
% Left: effect of damping ratio ζ
% ------------------------------------------------------------
wn_fixed = 2.0;
zetas_under = [0.1, 0.3, 0.6];
zetas_over  = [1.4, 2.0];

% ------------------------------------------------------------
% Right: effect of natural frequency ω_n
% ------------------------------------------------------------
zeta_fixed = 0.2;
wns = [1.0, 3.0, 5.0];

% ------------------------------------------------------------
% Plot (layout fixed, legend top-right)
% ------------------------------------------------------------
figure('Units','pixels','Position',[100 100 1300 450],'Color','w');
tl = tiledlayout(1,2,'TileSpacing','compact','Padding','compact');

% ===== Left panel: varying zeta =====
ax1 = nexttile(tl);
hold(ax1,'on');

for z = zetas_under
    y = y_response(t, z, wn_fixed, 1.0, 0.0);
    plot(ax1, t, y, 'LineWidth', 2, ...
        'DisplayName', sprintf('underdamped: $\\zeta=%.1f$', z));
end

% critically damped (highlight)
ycrit = y_response(t, 1.0, wn_fixed, 1.0, 0.0);
plot(ax1, t, ycrit, 'LineWidth', 3, ...
    'DisplayName', 'critical: $\zeta=1$');

for z = zetas_over
    y = y_response(t, z, wn_fixed, 1.0, 0.0);
    plot(ax1, t, y, 'LineWidth', 2, ...
        'DisplayName', sprintf('overdamped: $\\zeta=%.1f$', z));
end

yline(ax1, 0, 'LineWidth', 1, 'Color', [0 0 0], 'Alpha', 0.3);

text(ax1, 0.02, 0.95, 'transition at $\zeta=1$', ...
    'Units','normalized', 'Interpreter','latex', 'VerticalAlignment','top');

title(ax1, sprintf('Effect of damping ratio $\\zeta$ (fixed $\\omega_n=%.1f$)', wn_fixed), ...
    'Interpreter','latex');
xlabel(ax1, 't');
ylabel(ax1, 'y(t)');
grid(ax1,'on'); ax1.GridAlpha = 0.3;
box(ax1,'on');

% Legend: top-right (Matplotlib style)
legend(ax1, 'Interpreter','latex', 'FontSize',9, ...
    'Location','northeast');

% Padding AFTER legend
drawnow;
pad_axes(ax1, 0.08);

hold(ax1,'off');

% ===== Right panel: varying wn =====
ax2 = nexttile(tl);
hold(ax2,'on');

for wn = wns
    y = y_response(t, zeta_fixed, wn, 1.0, 0.0);
    plot(ax2, t, y, 'LineWidth', 2, ...
        'DisplayName', sprintf('$\\omega_n=%.1f$', wn));
end

title(ax2, sprintf('Effect of natural frequency $\\omega_n$ (fixed $\\zeta=%.1f$)', zeta_fixed), ...
    'Interpreter','latex');
xlabel(ax2, 't');
ylabel(ax2, 'y(t)');
grid(ax2,'on'); ax2.GridAlpha = 0.3;
box(ax2,'on');

% Legend: top-right
legend(ax2, 'Interpreter','latex', 'FontSize',9, ...
    'Location','northeast');

% Padding AFTER legend
drawnow;
pad_axes(ax2, 0.08);

hold(ax2,'off');


% ============================
% Local functions
% ============================

function y = y_response(t, zeta, wn, y0, yd0)
% Closed-form response for y¨ + 2 ζ ω_n y˙ + ω_n^2 y = 0
% with initial conditions y(0)=y0, y˙(0)=yd0.

    zeta = double(zeta);
    wn   = double(wn);

    if abs(zeta - 1.0) < 1e-12
        % critically damped
        C1 = y0;
        C2 = yd0 + wn * y0;
        y = (C1 + C2 .* t) .* exp(-wn .* t);
        return;
    end

    if zeta < 1.0
        % underdamped
        wd = wn * sqrt(1.0 - zeta^2);
        A = y0;
        B = (yd0 + zeta * wn * y0) / (wd + 1e-12);
        y = exp(-zeta * wn .* t) .* (A .* cos(wd .* t) + B .* sin(wd .* t));
        return;
    end

    % overdamped
    s = wn * sqrt(zeta^2 - 1.0);
    r1 = -wn * zeta + s;
    r2 = -wn * zeta - s;
    C2 = (yd0 - r1 * y0) / (r2 - r1 + 1e-12);
    C1 = y0 - C2;
    y = C1 .* exp(r1 .* t) + C2 .* exp(r2 .* t);
end

function pad_axes(ax, frac)
% Add Matplotlib-style padding to current axis limits.

    xl = xlim(ax);
    yl = ylim(ax);

    dx = xl(2) - xl(1);
    dy = yl(2) - yl(1);
    if dx == 0, dx = 1; end
    if dy == 0, dy = 1; end

    x_pad = frac * dx;
    y_pad = frac * dy;

    xlim(ax, [xl(1), xl(2) + x_pad]);
    ylim(ax, [yl(1) - y_pad, yl(2) + y_pad]);
end

